# Monitor ML runs live 

## Introduction

This guide will show you how to:

* Monitor training and evaluation metrics and losses live
* Monitor hardware resources during training

By the end of it, you will monitor your metrics, losses, and hardware live in Neptune!

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune-client tensorflow

## Create a basic training script

As an example I'll use a script that trains a Keras model on the MNIST dataset.

In [ ]:
from tensorflow import keras

# parameters
params = {
    "epoch_nr": 10,
    "batch_size": 256,
    "lr": 0.005,
    "momentum": 0.4,
    "use_nesterov": True,
    "unit_nr": 256,
    "dropout": 0.05,
}

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = keras.models.Sequential(
    [
        keras.layers.Flatten(),
        keras.layers.Dense(params["unit_nr"], activation=keras.activations.relu),
        keras.layers.Dropout(params["dropout"]),
        keras.layers.Dense(10, activation=keras.activations.softmax),
    ]
)

optimizer = keras.optimizers.SGD(
    lr=params["lr"],
    momentum=params["momentum"],
    nesterov=params["use_nesterov"],
)

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

## Initialize Neptune and create new run

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init_run(project="common/quickstarts", api_token=neptune.ANONYMOUS_API_TOKEN)

Click on the link above to open this run in Neptune.

For now it is empty but keep the tab with run open to see what happens next. 

**Few explanations**

In the above code You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/quickstarts' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init_run()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the `project` argument of the `init_run()` method.

For example:

```python
neptune.init_run(project="YOUR_WORKSPACE/YOUR_PROJECT", 
             api_token="YOUR_API_TOKEN")
```

## Add logging for metrics and losses

Since we are using Keras we'll create a Callback that logs metrics and losses after every epoch. 

In [ ]:
class NeptuneLogger(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            run[f"batch/{log_name}"].append(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            run[f"epoch/{log_name}"].append(log_value)

We need to pass it to the `callbacks` argument. 

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=params["epoch_nr"],
    batch_size=params["batch_size"],
    callbacks=[NeptuneLogger()],
)

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## See results live in the UI

Go to the `Charts` section of the Neptune app to see them. 

![image](https://neptune.ai/wp-content/uploads/docs-ml-runs-charts.png)

Neptune automatically logs the hardware consumption during the run. 

You can see it in the `Monitoring` section of the Neptune app. 

![image](https://neptune.ai/wp-content/uploads/docs-ml-runs-hardware.png)